In [1]:
! git init

Reinitialized existing Git repository in /Users/gaylejuntilla/MA3831_A3/.git/


In [13]:
!git add .
!git commit -m "Initialise abc_news.ipynb"

[main 7717188] Initialise abc_news.ipynb


In [14]:
!git push -u origin main

Enumerating objects: 12, done.
Counting objects: 100% (12/12), done.
Delta compression using up to 8 threads
Compressing objects: 100% (11/11), done.
Writing objects: 100% (11/11), 3.21 KiB | 3.21 MiB/s, done.
Total 11 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), done.
To https://github.com/gaylejuntilla/MA3831-A3.git
   cd64767..7717188  main -> main
branch 'main' set up to track 'origin/main'.


In [24]:
#pip install webdriver_manager

Note: you may need to restart the kernel to use updated packages.


In [28]:
# pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 3.6 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: attrs
    Found existing installation: attrs 23.1.0
    Uninstalling attrs-23.1.0:
      Successfully uninstalled attrs-23.1.0
Note: you may need to restart the kernel to use updated packages.


In [43]:
from selenium import webdriver
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
from datetime import datetime, timezone, timedelta
from selenium.webdriver.chrome.service import Service

class ABCNewsScraper:
    def __init__(self, start_url, cutoff_date="1 October 2024"):
        self.start_url = start_url
        self.cutoff_date = datetime.strptime(cutoff_date, "%d %B %Y")  # Convert string to datetime
        self.articles = pd.DataFrame(columns=['headline', 'title', 'date', 'content', 'url'])  # Store data
        # Initialize the service object for ChromeDriverManager
        service = Service(ChromeDriverManager().install())
        
        # Initialize WebDriver with service and options
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')  # Run headless (no UI)
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        
        self.driver = webdriver.Chrome(service=service, options=options) 
        
    def run(self):
        """Runs the scraper: loads page, extracts articles, stops at cutoff date."""
        self.driver.get(self.start_url)
        time.sleep(3)  # Let page load
        
        # Step 1: Load articles until the cutoff date
        self.load_articles_until_cutoff()
        
        # Step 2: Extract articles after loading all
        self.extract_articles()
        
        self.driver.quit()  # Close browser
        print(self.articles)
        #self.save_data_to_file()

    def load_articles_until_cutoff(self):
        """Keep loading more articles until the last article is older than the cutoff date."""
    
        while True:
            # Get all currently loaded articles
            articles = self.driver.find_elements(By.CSS_SELECTOR, 'ul[aria-labelledby="Latest election articles"] li')

            if not articles:
                print("No articles found on page. Stopping.")
                break

            # Find the date of the last article
            try:
                last_article = articles[-1]
                last_date_str = last_article.find_element(By.XPATH, ".//time[1]").get_attribute("datetime")
                last_article_date_utc = datetime.strptime(last_date_str, "%Y-%m-%dT%H:%M:%S.000Z")
                last_article_date = self.convert_to_brisbane_time(last_article_date_utc)

                # Stop loading if the last article's date is older than the cutoff
                if last_article_date < self.cutoff_date:
                    print("Reached cutoff date. Stopping 'Load More' clicks.")
                    break

            except Exception as e:
                print(f"Error extracting date from last article: {e}")
                break  # Stop in case of unexpected errors

            # Try clicking "Load More" to get more articles
            try:
                load_more_button = WebDriverWait(self.driver, 5).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[data-component="PaginationLoadMoreButton"]')))
                load_more_button.click()
                time.sleep(2)  # Wait for more articles to load

            except:
                print("No more 'Load More' button. Ending loading.")
                break
    
    def convert_to_brisbane_time(self, utc_datetime):
        """Converts a UTC datetime string to Brisbane time and formats it as 'DD Month YYYY'."""
    
        # Define UTC timezone
        utc_zone = timezone.utc

        # Define Brisbane timezone (UTC+10, no daylight savings)
        brisbane_zone = timezone(timedelta(hours=10))

        # Convert to Brisbane time
        brisbane_datetime = utc_datetime.replace(tzinfo=timezone.utc).astimezone(brisbane_zone)
        
        brisbane_datetime_naive = brisbane_datetime.replace(tzinfo=None)
        
        # Format as 'DD Month YYYY'
        return brisbane_datetime_naive
    
    def extract_articles(self):
        """Extracts article details: headline, date, content, and URL."""
        
        articles = self.driver.find_elements(By.CSS_SELECTOR, 'ul[aria-labelledby="Latest election articles"] li')

        for article in articles:
            try:
                
                # Extract date
                article_date_str = article.find_element(By.XPATH, ".//time[1][@datetime]").get_attribute('datetime')

                # Convert the UTC datetime string to a datetime object
                article_date_utc = datetime.strptime(article_date_str, "%Y-%m-%dT%H:%M:%S.000Z")

                # Format it to just Day, Month, and Year
                article_date = self.convert_to_brisbane_time(article_date_utc)
                
                # Stop if we've reached the cutoff date
                if article_date < self.cutoff_date:
                    break
                    
                # Extract the article URL
                link_element = article.find_element(By.TAG_NAME, 'a')
                article_url = link_element.get_attribute("href")

                # Extract headline
                headline = article.find_element(By.CSS_SELECTOR, "a[data-component='Link']").text 
                
                # Extract content
                self.driver.execute_script("window.open('');")  # Open new tab
                self.driver.switch_to.window(self.driver.window_handles[1])
                self.driver.get(article_url)
                time.sleep(3) # time to load article
                
                title = self.driver.find_element(By.CSS_SELECTOR, "h1[data-component='Typography']").text

                content_elements = self.driver.find_elements(By.CSS_SELECTOR, 'div[class*="ArticleRender_article"] p')
                content = " ".join([p.text for p in content_elements])

                # Close tab and switch back
                self.driver.close()
                self.driver.switch_to.window(self.driver.window_handles[0])

                # Add data to DataFrame
                self.articles.loc[len(self.articles)] = [headline, title, article_date, content, article_url]
                print(f"Collected: {headline}")
                
                time.sleep(10)

            except Exception as e:
                print(f"Error processing article: {e}")

    
    def save_data_to_file(self):
        """Saves the scraped data to a CSV file."""
        self.articles.to_csv("abc_news_articles.csv", index=False)
        print("Data saved to abc_news_articles.csv.")


In [48]:
!git add .
!git commit -m "IT WORKS (test)"
!git push

[main 5703832] IT WORKS (test)
 1 file changed, 368 insertions(+), 20 deletions(-)
Enumerating objects: 17, done.
Counting objects: 100% (17/17), done.
Delta compression using up to 8 threads
Compressing objects: 100% (15/15), done.
Writing objects: 100% (15/15), 9.49 KiB | 9.49 MiB/s, done.
Total 15 (delta 9), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (9/9), completed with 1 local object.
To https://github.com/gaylejuntilla/MA3831-A3.git
   851516c..5703832  main -> main


In [45]:
testcrawler = ABCNewsScraper("https://www.abc.net.au/news/elections/federal-election-2025", cutoff_date="24 March 2025")

In [46]:
testcrawler.run()

Reached cutoff date. Stopping 'Load More' clicks.
Collected: 'The good guy!': MP full of praise for himself in social media blunder
Collected: The WA seats that could decide Australia's next government
Collected: Dutton sees path to minority government, while both leaders declare they can win majority
Collected: Two SA seats will be hotly contested in the federal election
Collected: Why don't we have a set date for elections?
Collected: Experts poke holes in Peter Dutton's gas plan, saying 'the horse has bolted'
Collected: Plan to cut public servant jobs could be 'disastrous', Pocock says
Collected: The election has been called. What happens now?
Collected: Tasmania's state of play: the toss-up seat, the minister's challenge, and Labor's long shot
Collected: The race is on: Election campaign to be fought on cost of living, energy
Collected: Dutton's latest barrage in Coalition's 25-year war with itself on fuel excise
Collected: Compare the pair: Which major party's tax policy leaves yo

In [47]:
testcrawler.articles

,headline,title,date,content,url
0,'The good guy!': MP full of praise for himself...,Federal MP Andrew Gee blames staff impersonati...,2025-03-29 15:55:45,Independent federal MP Andrew Gee has blamed a...,https://www.abc.net.au/news/2025-03-29/andrew-...
1,The WA seats that could decide Australia's nex...,These Western Australian seats could decide wh...,2025-03-29 15:39:56,Western Australians — brace yourselves. Just w...,https://www.abc.net.au/news/2025-03-29/2025-fe...
2,"Dutton sees path to minority government, while...",Federal election 2025 live: Dutton sees path t...,2025-03-29 15:36:59,Prime Minister Anthony Albanese and Opposition...,https://www.abc.net.au/news/2025-03-29/federal...
3,Two SA seats will be hotly contested in the fe...,Federal election love descends on marginal sea...,2025-03-29 06:34:08,Voters in South Australia could be forgiven fo...,https://www.abc.net.au/news/2025-03-29/what-th...
4,Why don't we have a set date for elections?,When is the federal election? Why isn't it on ...,2025-03-29 09:22:28,Before the date of this year's election was of...,https://www.abc.net.au/news/2025-03-29/when-is...
5,"Experts poke holes in Peter Dutton's gas plan,...",More gas and lower prices 'years away' as expe...,2025-03-28 17:03:43,In the febrile environment of a federal electi...,https://www.abc.net.au/news/2025-03-28/experts...
6,Plan to cut public servant jobs could be 'disa...,ACT senators criticise Coalition for plan to c...,2025-03-28 14:57:47,When political parties plot where to send thei...,https://www.abc.net.au/news/2025-03-28/coaliti...
7,The election has been called. What happens now?,What date is the 2025 federal election? How to...,2025-03-28 09:43:16,The federal election has finally been called. ...,https://www.abc.net.au/news/2025-03-28/when-is...
8,"Tasmania's state of play: the toss-up seat, th...",Tasmania's state of play for the federal elect...,2025-03-29 09:17:20,It's historically difficult for election analy...,https://www.abc.net.au/news/2025-03-28/federal...
9,The race is on: Election campaign to be fought...,Campaign kicks off ahead of May 3 federal elec...,2025-03-28 15:29:07,Anthony Albanese has officially called the fed...,https://www.abc.net.au/news/2025-03-28/federal...
